# [Clustering4Ever](https://github.com/Clustering4Ever/Clustering4Ever) on [SparkNotebook](http://spark-notebook.io/) by [LIPN](https://lipn.univ-paris13.fr/) [A3](https://lipn.univ-paris13.fr/accueil/equipe/a3/) team

# Scala _K_-Means

In [ ]:
import smile.plot._
import scala.io.Source
import scala.collection.{immutable, mutable, parallel}
import clustering4ever.scala.clustering.kmeans.{KMeans, KMeansModel}
import clustering4ever.math.distances.scalar.{Euclidean, Cosine, Minkowski}
import clustering4ever.scala.indexes.ExternalIndexes
import clustering4ever.scala.indexes.NmiNormalizationNature._
import clustering4ever.util.ScalaImplicits._
import clustering4ever.scala.clusterizables.RealClusterizable
import clustering4ever.scala.vectorizables.RealVector

import smile.plot._
import scala.io.Source
import scala.collection.{immutable, mutable, parallel}
import clustering4ever.scala.clustering.kmeans.{KMeans, KMeansModel}
import clustering4ever.math.distances.scalar.{Euclidean, Cosine, Minkowski}
import clustering4ever.scala.indexes.ExternalIndexes
import clustering4ever.scala.indexes.NmiNormalizationNature._
import clustering4ever.util.ScalaImplicits._
import clustering4ever.scala.clusterizables.RealClusterizable
import clustering4ever.scala.vectorizables.RealVector


## Download dataset Aggregation

In [ ]:
:sh wget -P /tmp/ http://www.clustering4ever.org/Datasets/Aggregation/aggregation.csv
:sh wget -P /tmp/ http://www.clustering4ever.org/Datasets/Aggregation/labels

--2018-09-07 13:36:20--  http://www.clustering4ever.org/Datasets/Aggregation/aggregation.csv
Resolving www.clustering4ever.org (www.clustering4ever.org)... 62.210.16.62
Connecting to www.clustering4ever.org (www.clustering4ever.org)|62.210.16.62|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8063 (7.9K) [text/csv]
Saving to: ‘/tmp/aggregation.csv.9’

     0K .......                                               100%  365M=0s

2018-09-07 13:36:20 (365 MB/s) - ‘/tmp/aggregation.csv.9’ saved [8063/8063]

:sh: Scheme missing.
--2018-09-07 13:36:20--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2018-09-07 13:36:20--  http://www.clustering4ever.org/Datasets/Aggregation/labels
Reusing existing connection to www.clustering4ever.org:80.
HTTP request sent, awaiting response... 200 OK
Length: 1576 (1.5K)
Saving to: ‘/tmp/labels.10’

     0K .                                                     100

## Import data as ParArray[ArrayBuffer[Double]]

In [ ]:
val datasetSize = 2000000
val dim = 100
val path = "/tmp/aggregation.csv"
val useAggregationDS = false

val rawData = if( useAggregationDS ) Source.fromFile(path).getLines.map( x => mutable.ArrayBuffer(x.split(",").map(_.toDouble):_*) ).toBuffer
  else mutable.ArrayBuffer.fill(datasetSize)(mutable.ArrayBuffer.fill(dim)(scala.util.Random.nextDouble))

val parData = rawData.zipWithIndex.par.map{ case (v, id) => new RealClusterizable(id.toLong, new RealVector(v)) }

val groundTruePath = "/tmp/labels"

datasetSize: Int = 2000000
dim: Int = 100
path: String = /tmp/aggregation.csv
useAggregationDS: Boolean = false
rawData: scala.collection.mutable.Buffer[scala.collection.mutable.ArrayBuffer[Double]] = ArrayBuffer(ArrayBuffer(0.4127021608244922, 0.015072342651154869, 0.10799190350406374, 0.10970637312396847, 0.1569008905498509, 0.8597422669616767, 1.2960782260285075E-4, 0.9853396805660675, 0.2018103125715781, 0.14642448268847053, 0.8623201639574947, 0.5251123554154499, 0.5099008600171878, 0.8963921306535552, 0.42262032729746724, 0.5290781941825453, 0.3367476330990349, 0.3466161572952129, 0.8823357220995842, 0.06588807078536485, 0.10660155964586138, 0.7844106895827907, 0.18356579593947964, 0.9180716008198188, 0.025498839682826735, 0.9217484800890267, 0.49727395376698014, 0.725878583145658...

## Parameters 

In [ ]:
val k = 7
val iterMax = 40
val epsilon = 0.5
// True defines the true Euclidean with square root, set it to false to avoid the square
// val metric0 = new Euclidean[Array[Double]](true)
val metric1 = new Euclidean[mutable.ArrayBuffer[Double]](true)

// Minkowski(p) where p is the Minkowski parameter
val metric2 = new Minkowski[mutable.ArrayBuffer[Double]](2)

k: Int = 7
iterMax: Int = 40
epsilon: Double = 0.5
metric1: clustering4ever.math.distances.scalar.Euclidean[scala.collection.mutable.ArrayBuffer[Double]] = Euclidean distance with root applied
metric2: clustering4ever.math.distances.scalar.Minkowski[scala.collection.mutable.ArrayBuffer[Double]] = clustering4ever.math.distances.scalar.Minkowski@6e6796b1


## Run and measure the algorithm time

In [ ]:
val t1 = System.nanoTime

val model = KMeans.run[
              Long,
              mutable.ArrayBuffer[Double],
              mutable.ArrayBuffer[Double],
              RealClusterizable[Long, mutable.ArrayBuffer[Double], mutable.ArrayBuffer[Double]],
              Euclidean[mutable.ArrayBuffer[Double]]
              ](
                parData,
                k,
                epsilon,
                iterMax,
                metric1
              )

val t2 = System.nanoTime

(t2 - t1) / 1000000000D

t1: Long = 15011508962559
model: clustering4ever.scala.clustering.kmeans.KMeansModel[Long,scala.collection.mutable.ArrayBuffer[Double],scala.collection.mutable.ArrayBuffer[Double],clustering4ever.scala.clusterizables.RealClusterizable[Long,scala.collection.mutable.ArrayBuffer[Double],scala.collection.mutable.ArrayBuffer[Double]],clustering4ever.math.distances.scalar.Euclidean[scala.collection.mutable.ArrayBuffer[Double]]] = clustering4ever.scala.clustering.kmeans.KMeansModel@24ed7f00
t2: Long = 15024893349045
res6: Double = 13.384386486


13.384386486

# Compare algorithm between ParArray and ArraySeq container

In [ ]:
val t1 = System.nanoTime

KMeans.run[Long,
    mutable.ArrayBuffer[Double],
    mutable.ArrayBuffer[Double],
    RealClusterizable[Long, mutable.ArrayBuffer[Double], mutable.ArrayBuffer[Double]],
    Euclidean[mutable.ArrayBuffer[Double]]
    ](parData, k, epsilon, iterMax, metric1)

val t2 = System.nanoTime
val seqData = parData.seq
val t3 = System.nanoTime

KMeans.run[Long,
    mutable.ArrayBuffer[Double],
    mutable.ArrayBuffer[Double],
    RealClusterizable[Long, mutable.ArrayBuffer[Double], mutable.ArrayBuffer[Double]],
    Euclidean[mutable.ArrayBuffer[Double]]
    ](seqData, k, epsilon, iterMax, metric1)

val t4 = System.nanoTime



(t4 - t3).toDouble / (t2 - t1)

t1: Long = 15025863841986
t2: Long = 15035361233137
seqData: scala.collection.mutable.Seq[clustering4ever.scala.clusterizables.RealClusterizable[Long,scala.collection.mutable.ArrayBuffer[Double],scala.collection.mutable.ArrayBuffer[Double]]] = ArraySeq(RealClusterizable(0,clustering4ever.scala.vectorizables.RealVector@30b900c6,List(),2147483647), RealClusterizable(1,clustering4ever.scala.vectorizables.RealVector@21a0c162,List(),2147483647), RealClusterizable(2,clustering4ever.scala.vectorizables.RealVector@73721b8f,List(),2147483647), RealClusterizable(3,clustering4ever.scala.vectorizables.RealVector@4adf031a,List(),2147483647), RealClusterizable(4,clustering4ever.scala.vectorizables.RealVector@329895b7,List(),2147483647), RealClusterizable(5,clustering4ever.scala.vectorizables.RealVect...

4.393892996457886

## Different ways to apply model to datasets

In [ ]:
val clusterized11 = rawData.map( v => (model.centerPredict(v), v) )
val clusterized12 = model.centerPredict(rawData)

val clusterized21 = parData.map( clustbl => (model.centerPredict(clustbl.vector), clustbl) )
val clusterized22 = model.centerPredict(parData)

clusterized11: scala.collection.mutable.Buffer[(model.ClusterID, scala.collection.mutable.ArrayBuffer[Double])] = ArrayBuffer((0,ArrayBuffer(0.4127021608244922, 0.015072342651154869, 0.10799190350406374, 0.10970637312396847, 0.1569008905498509, 0.8597422669616767, 1.2960782260285075E-4, 0.9853396805660675, 0.2018103125715781, 0.14642448268847053, 0.8623201639574947, 0.5251123554154499, 0.5099008600171878, 0.8963921306535552, 0.42262032729746724, 0.5290781941825453, 0.3367476330990349, 0.3466161572952129, 0.8823357220995842, 0.06588807078536485, 0.10660155964586138, 0.7844106895827907, 0.18356579593947964, 0.9180716008198188, 0.025498839682826735, 0.9217484800890267, 0.49727395376698014, 0.7258785831456588, 0.3561337376802901, 0.868480895194272, 0.7921421195513476, 0.4226061713613388, 0....

## Plot clustering results

In [ ]:
val rawData = clusterized22.map( rc => rc.vector.toArray ).toArray
val labels = clusterized22.map( rc => rc.clusterID ).toArray
plot(rawData, labels, '*', Palette.COLORS)

java.lang.IllegalArgumentException: Invalid data dimension: 100
  at smile.plot.ScatterPlot.plot(ScatterPlot.java:514)
  at smile.plot.ScatterPlot.plot(ScatterPlot.java:484)
  at smile.plot.Operators$class.plot(Operators.scala:85)
  at smile.plot.package$.plot(package.scala:23)
  ... 69 elided


## Inspect performance metrics

In [ ]:
val trueLabels = Source.fromFile(groundTruePath).getLines.map(_.toInt).toBuffer

val (_, trueLabelsFrom0) = ExternalIndexes.prepareLabels(trueLabels)

val predicts = clusterized22.map(_.clusterID).seq

val nmi = ExternalIndexes.nmi(trueLabelsFrom0, predicts, SQRT)

nmi

java.lang.IllegalArgumentException: requirement failed
  at scala.Predef$.require(Predef.scala:212)
  at clustering4ever.scala.indexes.ExternalIndexes.mutualInformationInternal(ExternalIndexes.scala:16)
  at clustering4ever.scala.indexes.ExternalIndexes$.nmi(ExternalIndexes.scala:64)
  ... 69 elided
